## Image classification with Convolutional Neural Networks

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

from pathlib import Path

In [3]:
if os.name == 'nt':
    PATH = str(Path.home()) + "\\data\\dogscats\\"
else:
    PATH = "data/dogscats/"
    
sz=224
arch=vgg16
bs=64

In [4]:
# Uncomment the below if you need to reset your precomputed activations
if os.name == 'nt':
    ## warning: will remove everything under the following path (windows):
    #delPath = f"{PATH}tmp\\"
    #!rmdir /S /Q "{delPath}"
    ;
else:
    #!rm -rf {PATH}tmp
    ;

In [5]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))

In [6]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [7]:
%time learn.fit(0.01, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[0.      0.06019 0.03293 0.98754]
[1.      0.04656 0.03226 0.98932]
[2.      0.05482 0.02826 0.9909 ]

Wall time: 13.4 s


In [8]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [9]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [10]:
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[0.      0.05199 0.03414 0.98794]
[1.      0.05799 0.02832 0.99051]



In [11]:
learn.precompute=False

In [12]:
%time learn.fit(1e-2, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[0.      0.05396 0.02959 0.98932]

Wall time: 2min 3s


In [13]:
learn.unfreeze()

In [14]:
lr=np.array([1e-4,1e-3,1e-2])

In [15]:
%time learn.fit(lr, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[0.      0.05801 0.03333 0.98457]

Wall time: 4min 35s


In [16]:
%time learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[0.      0.05681 0.02795 0.98873]
[1.      0.04353 0.02558 0.99031]
[2.      0.03442 0.02447 0.9913 ]
[3.      0.04071 0.02398 0.9913 ]
[4.      0.0416  0.02363 0.99229]
[5.      0.02514 0.02198 0.99268]
[6.      0.02802 0.022   0.99268]

Wall time: 32min 18s


In [17]:
%time learn.fit(lr, 3, cycle_len=3)

HBox(children=(IntProgress(value=0, description='Epoch', max=9), HTML(value='')))

[0.      0.0249  0.02163 0.99209]
[1.      0.02683 0.02171 0.99208]
[2.      0.02051 0.02141 0.99229]
[3.      0.02703 0.02456 0.99148]
[4.      0.02229 0.02236 0.99208]
[5.      0.01826 0.02308 0.99308]
[6.      0.01773 0.02163 0.99229]
[7.      0.01715 0.02301 0.99209]
[8.      0.02089 0.02371 0.99109]

Wall time: 41min 26s


In [18]:
%time log_preds_augmented,y = learn.TTA()
log_preds = log_preds_augmented.mean(axis=0)
accuracy(log_preds, y)

Wall time: 1min 25s


0.9938283894087199